# Persistence divergence on Torus

In this example, we review persistence divergence of autonomoous wheelchairs moving on a square with periodic boundary conditions (i.e. a torus)

Firs of all, we import some necessary modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl
import iblofunmatch.inter as ibfm
output_dir="output"
plots_dir = "plots/torus_cross/"
import os

os.makedirs(plots_dir, exist_ok=True)

from perdiver.distances import *
import perdiver.perdiver as perdiver

In this experiment, we see a big change in dynamics when changing agents from $31$ to $33$. The first case follows a very ordered movement. On the second case, the movement is much less ordered.

In [ ]:
from navground import sim, core
side=15
behaviour = "ORCA"
yaml = f"""
steps: 1000
time_step: 0.1
record_pose: true
record_twist: true
runs: 1
scenario:
  type: CrossTorus
  agent_margin: 0.2
  side: {side}
  target_margin: 0.6
  tolerance: 0.4
  groups:   
    -
      type: thymio
      number: 29
      control_period: 0.1
      behavior:
        type: {behaviour}
        safety_margin: 0.2 
        horizon: 3
        barrier_angle: 1
      radius: 0.2
      kinematics:
        type: 2WDiff
        max_speed: 1.0
        wheel_axis: 2
      state_estimation:
        type: Bounded
        range: 2.0 
"""
experiment = sim.load_experiment(yaml)
experiment.run()
# experiment.record_config.set_all(True)

# experiment.record_config.set_all(True)
# path = "recorded_experiment.h5"
# os.remove(path)
# experiment.run(keep=False, data_path=path)
# recorded_experiment = sim.RecordedExperiment(path)

Now, let us read the velocities and positions of wheelchairs on the simulation.

In [ ]:
# recorded_experiment = sim.RecordedExperiment(path)
# run = recorded_experiment.runs[0]
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

Let us visualise the experiment in both situations. First, we plot the case when the wheelchairs stabilise into constant speed trajectories.

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
from navground.sim.ui.video import record_video, record_video_from_run

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f


fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    if isinstance(entity, sim.Agent):
        return {'fill': fill_map(entity.behavior.efficacy)}
    return {}
    
record_video(f"{plots_dir}{behaviour}torus_ordered_exp.mp4", run.world, time_step=0.1, duration=60.0, factor=6.0,
             bounds=((-side*0.1, -side*0.1), (side*1.1, side*1.1)), decorate=f, width=500)

# run.world = ((-side*0.1, -side*0.1), (side*1.1, side*1.1))
# record_video_from_run(f"{behaviour}torus_ordered_exp.mp4", run, factor= 1.0, fps= 30, decorate=f, width=500, from_time=0, to_time=20)

In [ ]:
from IPython.display import Video

Video(f"{plots_dir}{behaviour}torus_ordered_exp.mp4", width=200)

In [ ]:
# from IPython.display import Video

# Video("torus_messy_exp.mp4", width=200)

Next, let us see the impact on persistence divergence diagrams.

In [ ]:
weight = 3

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(4,4))
Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_2D_torus(ps, twists, 800, 100, weight, side)
match_diagram = perdiver.get_matching_diagram(Dist_X, Dist_Y)
perdiver.plot_matching_diagram(match_diagram, ax, color="blue")

Plot persistence divergence over first 2500 steps.

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
steplist = list(range(2000, 2500, 50))
shift_time=30
for idx, start_step_traj in enumerate(steplist):
    Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_2D_torus(ps, twists, start_step_traj, shift_time, weight, side)
    perdiver.plot_matching_diagram( Dist_X, Dist_Y, Dist_Z, ax, color=mpl.colormaps["GnBu"](idx/len(steplist)))

norm = mpl.colors.Normalize(vmin=steplist[0], vmax=steplist[-1])
cmap = mpl.colormaps["GnBu"]
mappable = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
plt.colorbar(mappable=mappable, ax=ax)
plt.savefig(f"{plots_dir}{behaviour}_perdiver.png")
# plt.savefig("perdiver_diagram_messy.png")

In [ ]:
start_step_list = list(range(898,1340, 100))
fig, ax = plt.subplots(ncols=len(start_step_list), figsize=(6*(len(start_step_list)),6))
xmax, ymax = 0,0
for idx_start, start_step in enumerate(start_step_list):
    shift_time=40
    steplist = list(range(start_step, start_step+shift_time, 10))
    for idx, start_step_traj in enumerate(steplist):
        Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_2D_torus(ps, twists, start_step_traj, shift_time, weight, side)
        perdiver.plot_matching_diagram( Dist_X, Dist_Y, Dist_Z, ax[idx_start], color=mpl.colormaps["GnBu"](idx/len(steplist)))
    ax[idx_start].set_title(f"{start_step}")
    xmax = max(xmax, ax[idx_start].get_xlim()[1])
    ymax = max(ymax, ax[idx_start].get_ylim()[1])

for idx_start, start_step in enumerate(start_step_list):
    ax[idx_start].plot([0,max(xmax, ymax)], [0, max(xmax, ymax)], color="gray")
    ax[idx_start].set_xlim(0,xmax)
    ax[idx_start].set_ylim(0,ymax)

In [ ]:
np.unique(np.nonzero(run.safety_violations)[0])

In [ ]:
start_step=730
shift_time =40
np.sum(run.safety_violations[start_step:start_step+shift_time])

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
start_step=900
shift_time =40

Dist_X, Dist_Y, Dist_Z = compute_distance_matrices_trajectories_2D_torus(ps, twists, start_step, shift_time, weight, side)
perdiver.plot_matching_diagram_trajectories(
    Dist_X, Dist_Y, Dist_Z, ax, color="blue",
    print_barcode_n_reps=True
)

In [ ]:
X_seq = ps[list(range(start_step, start_step+shift_time+1, 2))]
len(X_seq)
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_aspect("equal")
perdiver.plot_sequence(X_seq, ax, mark_points=[15,27], color="yellow")

# Check out persistence divergence over time.

In [ ]:
steps_list = list(range(5000, 6000, 5))
shift_step = 100
divergence_arr, Z_barcodes_arr = perdiver.compute_divergence_Z_arrays(ps, twists, steps_list, shift_step, weight, side)

In [ ]:
def plot_divergence_array(div_arr, steps_list, ax, vmin=None, vmax=None):
    if vmin is None:
        vmin = np.min(div_arr)
    if vmax is None:
        vmax = np.max(div_arr)
    mapable = ax.imshow(div_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(steps_list[0], steps_list[-1], 0, div_arr.shape[0]))
    plt.colorbar(mapable)

In [ ]:
fig, ax = plt.subplots(figsize=(15,2))
plot_divergence_array(divergence_arr, steps_list, ax)
ax.set_title("Persistence Divergence")
plt.tight_layout()
plt.savefig("torus_cross_divergence_diag_hist.png")

In [ ]:
fig, ax = plt.subplots(figsize=(15,2))
plot_divergence_array(Z_barcodes_arr, steps_list, ax)
ax.set_title("Z barcode evolution")
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
cumulative_arr = perdiver.compute_cumulative_array(divergence_arr)
plot_divergence_array(cumulative_arr, steps_list, ax)
ax.set_title("Cumulative Persistence Divergence")
plt.tight_layout()

In [ ]:
prev_div_min = np.min(divergence_arr)
prev_div_max = np.max(divergence_arr)
prev_cdiv_min = np.min(cumulative_arr)
prev_cdiv_max = np.max(cumulative_arr)

#  Check again in another dataset.

In [ ]:
fig, ax = plt.subplots(figsize=(15,2))
plot_divergence_array(divergence_arr, steps_list, ax, vmin=prev_div_min, vmax=prev_div_max)
ax.set_title("cross_on_torus")
plt.tight_layout()
plt.savefig("torus_cross_divergence_diag_hist.png")

In [ ]:
fig, ax = plt.subplots(figsize=(15,2))
plot_divergence_array(Z_barcodes_arr, steps_list, ax)
ax.set_title("Z_barcode_cross_on_torus")
plt.tight_layout()

In [ ]:
# def compute_cumulative_array(div_arr):
#     cumulative_list = []
#     for j, divergence in enumerate(div_arr.transpose()):
#         cumulative = []
#         for i in range(div_arr.shape[0]):
#             cumulative.append(np.sum(divergence[:i+1]))
#         cumulative_list.append(cumulative)
#     return np.array(cumulative_list).transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
cumulative_arr = perdiver.compute_cumulative_array(divergence_arr)
plot_divergence_array(cumulative_arr, steps_list, ax, vmin=prev_cdiv_min, vmax=prev_cdiv_max)
ax.set_title("cumulative")
plt.tight_layout()

In [ ]:
# y_max = np.max(Z_barcodes_arr)
# y_min = np.min(Z_barcodes_arr)
# y_range = np.linspace(y_min, y_max, 40)
# combinations_list = []
# for y in y_range:
#     combinations_row = [] 
#     for j, column in enumerate(Z_barcodes_arr.transpose()):
#         idx = np.sum(Z_barcodes_arr[:,j] < y)-1
#         if idx < 0:
#             combinations_row.append(0)
#         else:
#             combinations_row.append(cumulative_arr[idx,j])
#     combinations_list.append(combinations_row)
# combinations_arr = np.array(combinations_list)

In [ ]:
# fig, ax = plt.subplots(figsize=(10,2))
# vmax = np.max(combinations_arr)
# vmin = np.min(combinations_arr)
# mapable = ax.imshow(combinations_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(steps_list[0], steps_list[-1], y_max, 0))
# ax.set_title("combinations")
# plt.colorbar(mapable)
# plt.tight_layout()

# Record simulation Video

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx

def linear_map(a, b, cmap):
    c = cmx.ScalarMappable(norm=colors.Normalize(vmin=a, vmax=b), cmap=cmap)  
    def f(v):
        r, g, b, _ = c.to_rgba(v)
        return f"#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}"
    return f

fill_map = linear_map(0.0, 1.0, cmap=cmx.RdYlGn)

def f(entity):
    if isinstance(entity, sim.Agent):
        return {'fill': fill_map(entity.behavior.efficacy)}
    return {}

In [ ]:
from navground.sim.ui.video import record_video

bounds = (np.array([-1,-1]), np.array([12,12]))
record_video("first_torus_cross_exp.mp4", run.world, time_step=0.1, duration=60, factor=5.0, decorate=f, width=300, display_shape=True, fps=30, bounds=bounds)

In [ ]:
from IPython.display import Video

Video("first_torus_cross_exp.mp4", width=300)